In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Replace the following placeholders with your actual database credentials
db_username = 'root'
db_password = '@Work00'
db_host = '127.0.0.1'
db_port = '8080'
db_name = 'gdb023'

# Create a database connection
engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

def execute_query(query):
    with engine.connect() as connection:
        result = pd.read_sql(query, connection)
    return result

# 1. Provide the list of markets in which customer "Atliq Exclusive" operates its business in the APAC region.
query1 = """
SELECT DISTINCT market 
FROM dim_customer 
WHERE customer = 'Atliq Exclusive' 
AND region = 'APAC';
"""
result1 = execute_query(query1)
print("Query 1 Result:\n", result1)

# 2. Percentage of unique product increase in 2021 vs. 2020.
query2 = """
WITH product_counts AS (
    SELECT COUNT(DISTINCT product_code) AS unique_products_2020 
    FROM fact_sales_monthly 
    WHERE fiscal_year = 2020 
    UNION ALL 
    SELECT COUNT(DISTINCT product_code) AS unique_products_2021 
    FROM fact_sales_monthly 
    WHERE fiscal_year = 2021
)
SELECT 
    (SELECT unique_products_2020 FROM product_counts WHERE unique_products_2020 IS NOT NULL) AS unique_products_2020,
    (SELECT unique_products_2021 FROM product_counts WHERE unique_products_2021 IS NOT NULL) AS unique_products_2021,
    ((SELECT unique_products_2021 FROM product_counts WHERE unique_products_2021 IS NOT NULL) - 
    (SELECT unique_products_2020 FROM product_counts WHERE unique_products_2020 IS NOT NULL)) * 100.0 / 
    (SELECT unique_products_2020 FROM product_counts WHERE unique_products_2020 IS NOT NULL) AS percentage_chg;
"""
result2 = execute_query(query2)
print("Query 2 Result:\n", result2)

# 3. Report with all the unique product counts for each segment sorted in descending order.
query3 = """
SELECT 
    segment, 
    COUNT(DISTINCT product_code) AS product_count 
FROM dim_product 
GROUP BY segment 
ORDER BY product_count DESC;
"""
result3 = execute_query(query3)
print("Query 3 Result:\n", result3)

# 4. Segment with the most increase in unique products in 2021 vs 2020.
query4 = """
WITH product_counts AS (
    SELECT 
        segment,
        COUNT(DISTINCT CASE WHEN fiscal_year = 2020 THEN product_code END) AS product_count_2020,
        COUNT(DISTINCT CASE WHEN fiscal_year = 2021 THEN product_code END) AS product_count_2021
    FROM fact_sales_monthly 
    JOIN dim_product USING (product_code)
    GROUP BY segment
)
SELECT 
    segment,
    product_count_2020,
    product_count_2021,
    (product_count_2021 - product_count_2020) AS difference
FROM product_counts
ORDER BY difference DESC
LIMIT 1;
"""
result4 = execute_query(query4)
print("Query 4 Result:\n", result4)

# 5. Products with the highest and lowest manufacturing costs.
query5 = """
(SELECT 
    product_code, 
    product, 
    manufacturing_cost 
FROM fact_manufacturing_cost 
JOIN dim_product USING (product_code)
ORDER BY manufacturing_cost DESC 
LIMIT 1)
UNION ALL
(SELECT 
    product_code, 
    product, 
    manufacturing_cost 
FROM fact_manufacturing_cost 
JOIN dim_product USING (product_code)
ORDER BY manufacturing_cost ASC 
LIMIT 1);
"""
result5 = execute_query(query5)
print("Query 5 Result:\n", result5)

# 6. Top 5 customers who received an average high pre_invoice_discount_pct for 2021 in the Indian market.
query6 = """
SELECT 
    dc.customer_code, 
    dc.customer, 
    AVG(fpd.pre_invoice_discount_pct) AS average_discount_percentage
FROM fact_pre_invoice_deductions fpd
JOIN dim_customer dc ON fpd.customer_code = dc.customer_code
WHERE fpd.fiscal_year = 2021 AND dc.market = 'India'
GROUP BY dc.customer_code, dc.customer
ORDER BY average_discount_percentage DESC
LIMIT 5;
"""
result6 = execute_query(query6)
print("Query 6 Result:\n", result6)

# 7. Gross sales amount for "Atliq Exclusive" for each month.
query7 = """
SELECT 
    DATE_FORMAT(date, '%Y-%m') AS Month,
    YEAR(date) AS Year,
    SUM(sold_quantity * gross_price) AS Gross_sales_Amount
FROM fact_sales_monthly fsm
JOIN dim_customer dc ON fsm.customer_code = dc.customer_code
JOIN fact_gross_price fgp ON fsm.product_code = fgp.product_code AND fsm.fiscal_year = fgp.fiscal_year
WHERE dc.customer = 'Atliq Exclusive'
GROUP BY Year, Month
ORDER BY Year, Month;
"""
result7 = execute_query(query7)
print("Query 7 Result:\n", result7)

# 8. Quarter of 2020 with the maximum total_sold_quantity.
query8 = """
WITH quarterly_sales AS (
    SELECT 
        QUARTER(date) AS Quarter,
        SUM(sold_quantity) AS total_sold_quantity
    FROM fact_sales_monthly
    WHERE fiscal_year = 2020
    GROUP BY QUARTER(date)
)
SELECT 
    CONCAT('Q', Quarter) AS Quarter,
    total_sold_quantity
FROM quarterly_sales
ORDER BY total_sold_quantity DESC
LIMIT 1;
"""
result8 = execute_query(query8)
print("Query 8 Result:\n", result8)

# 9. Channel with more gross sales in 2021 and the percentage of contribution.
query9 = """
WITH total_sales AS (
    SELECT 
        SUM(sold_quantity * gross_price) AS total_gross_sales
    FROM fact_sales_monthly fsm
    JOIN fact_gross_price fgp ON fsm.product_code = fgp.product_code AND fsm.fiscal_year = fgp.fiscal_year
    WHERE fsm.fiscal_year = 2021
)
SELECT 
    channel,
    SUM(fsm.sold_quantity * fgp.gross_price) / 1000000 AS gross_sales_mln,
    (SUM(fsm.sold_quantity * fgp.gross_price) / ts.total_gross_sales) * 100 AS percentage
FROM fact_sales_monthly fsm
JOIN dim_customer dc ON fsm.customer_code = dc.customer_code
JOIN fact_gross_price fgp ON fsm.product_code = fgp.product_code AND fsm.fiscal_year = fgp.fiscal_year
JOIN total_sales ts
WHERE fsm.fiscal_year = 2021
GROUP BY channel
ORDER BY gross_sales_mln DESC;
"""
result9 = execute_query(query9)
print("Query 9 Result:\n", result9)

# 10. Top 3 products in each division with high total_sold_quantity in 2021.
query10 = """
WITH ranked_products AS (
    SELECT 
        dp.division,
        fsm.product_code,
        dp.product,
        SUM(fsm.sold_quantity) AS total_sold_quantity,
        RANK() OVER (PARTITION BY dp.division ORDER BY SUM(fsm.sold_quantity) DESC) AS rank_order
    FROM fact_sales_monthly fsm
    JOIN dim_product dp ON fsm.product_code = dp.product_code
    WHERE fsm.fiscal_year = 2021
    GROUP BY dp.division, fsm.product_code, dp.product
)
SELECT 
    division,
    product_code,
    product,
    total_sold_quantity,
    rank_order
FROM ranked_products
WHERE rank_order <= 3
ORDER BY division, rank_order;
"""
result10 = execute_query(query10)
print("Query 10 Result:\n", result10)

# Close the database connection
engine.dispose()


OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'Work00@127.0.0.1' ([Errno 11003] getaddrinfo failed)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [14]:
import subprocess

# Define your MySQL credentials and file details
db_username = 'root'
db_password = '@Work00'
db_host = '127.0.0.1'
db_port = '8080'
db_name = 'gdb023'
sql_file_path = 'atliq_hardware_db.sql'  # Update this path to the location of your .sql file

# Command to execute
command = f"mysql -u {db_username} -p{db_password} -h {db_host} -P {db_port} {db_name} < {sql_file_path}"

try:
    # Execute the command
    subprocess.run(command, shell=True, check=True)
    print("SQL file imported successfully.")
except subprocess.CalledProcessError as e:
    print(f"Error during SQL file import: {e}")


Error during SQL file import: Command 'mysql -u root -p@Work00 -h 127.0.0.1 -P 8080 gdb023 < atliq_hardware_db.sql' returned non-zero exit status 1.
